In [8]:
import pandas as pd
from geopy.geocoders import Nominatim
import re
import folium

In [11]:
df = pd.read_excel('members.xlsx', header=1)

In [12]:
df.shape

(198, 11)

In [13]:
df = df[['Medlemstype', 'Startdato for aktuel medlemstype', 'Vej, husnr. og evt. etage', 'Postnummer og by']]

In [14]:
df.head(50)

,Medlemstype,Startdato for aktuel medlemstype,"Vej, husnr. og evt. etage",Postnummer og by
0,Medlem,2011-05-26,"Griffenfeldsgade 50, 3..",2200 København N
1,Medlem,2022-01-01,"Hamletsgade 8, 3. 121",2200 København N
2,Medlem,2001-11-20,"Sankt Hans Torv 3, 4tv.",2200 København N
3,Medlem,2018-07-30,"Julius Bloms Gade 3, th.",2200 København N
4,Medlem,2019-05-05,"Ørnevej 4, 1, tv.",2400 København NV
5,Medlem,2019-07-29,"Guldbergs Plads 10, 4. th..",2200 København N
6,Medlem,2024-01-01,"Borgmestervangen 31, 23. 5",2200 København N
7,Medlem,2014-02-18,"Blegdamsvej 42, 2th.",2200 København N
8,Medlem,1980-09-08,"Bogtrykkervej 16, 3tv.",2400 København NV
9,Medlem,2023-01-01,"Krogerupgade 7, 1.",2200 København N


In [15]:
df.to_csv('addresses.csv', header=True, index=False)

In [9]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium


# Combine the address columns into one
df['Full Address'] = df['Vej, husnr. og evt. etage'] + ', ' + df['Postnummer og by']

# Clean the address from unnecessary characters
def clean_address(address):
    address = re.sub(r'\.\.', '', address)  # Remove double dots
    address = re.sub(r'[^\w\s,]', '', address)  # Remove any non-alphanumeric characters
    return address

df['Cleaned Address'] = df['Full Address'].apply(clean_address)

# Initialize geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Function to geocode
def geocode(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return None
    except:
        return None

# Apply the geocode function
df['Coordinates'] = df['Cleaned Address'].apply(geocode)

# Drop rows where geocoding failed
df = df.dropna(subset=['Coordinates'])

# Separate latitude and longitude
df[['Latitude', 'Longitude']] = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)

# Drop the 'Coordinates' and 'Cleaned Address' columns as they are no longer needed
df = df.drop(columns=['Coordinates', 'Cleaned Address'])

# Display the DataFrame with the new Latitude and Longitude columns
print(df)

# Step 2: Creating the interactive map with folium
# Initialize the map centered around Copenhagen
m = folium.Map(location=[55.6761, 12.5683], zoom_start=12)

# Add points to the map
for idx, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Full Address']).add_to(m)

# Save the map to an HTML file
m.save('map.html')

# If you want to display the map in a Jupyter notebook:
m


                             Medlemstype Startdato for aktuel medlemstype  \
11                                Medlem                       2024-01-01   
13                                Medlem                       2023-01-01   
22                                Medlem                       2024-01-01   
30                                Medlem                       2023-01-01   
40                                Medlem                       2024-01-01   
42                                Medlem                       2008-03-13   
45                                Medlem                       2014-02-19   
49                                Medlem                       2023-01-01   
51                                Medlem                       2005-12-20   
58                                Medlem                       2023-01-01   
59                                Medlem                       2011-09-14   
69                                Medlem                       2012-06-15   